In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Link to Data:
#https://zindi.africa/hackathons/umojahack-1-saeon-identifying-marine-invertebrates/data

In [ ]:
sub=pd.read_csv('../input/simi2020/UmojaHack#1_SAEON_Identifying_marine_invertebrates/UmojaHack#1_SAEON_Identifying_marine_invertebrates/SampleSubmission .csv')

In [ ]:
classes=sub.columns[1:]


In [ ]:
%matplotlib inline
import cv2
import os
import math
import copy
import matplotlib.pyplot as plt
import random
from os import listdir
from os.path import isfile, join
import torchvision 
import torch 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
# import skimage
from skimage.io import imread

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
def set_seed(seed = 1234):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    #random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [ ]:
root_path='../input/simi2020/UmojaHack#1_SAEON_Identifying_marine_invertebrates/UmojaHack#1_SAEON_Identifying_marine_invertebrates/'
dirtr='train_small/train_small/'
dirts='test_small/test_small/'
i=0
label_map={}
l=[]
for fold in os.listdir(root_path + dirtr):
    l.append(fold)
l.sort()
for i in range(137):
    label_map[l[i]] = i

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.resnet50(pretrained=True)

In [ ]:
for layer, param in model.named_parameters():
    
    if 'layer4' not in layer:
        param.requires_grad = False

from torch import nn
fc_inputs = model.fc.in_features
 
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(fc_inputs, 137),
    #torch.nn.LogSoftmax(dim=1)
)

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

In [ ]:
dataset = datasets.ImageFolder(root=root_path + dirtr, transform=train_transforms)


In [ ]:
data=pd.DataFrame(dataset.targets,columns=['datatargets'])

In [ ]:
data.hist()

In [ ]:
batch_size = 32

#random_seed= 42
targets = dataset.targets

# stratified split for validation
train_idx, valid_idx= train_test_split(
    np.arange(len(targets)),
    test_size=0.2,
    shuffle=True,
    stratify=targets)
train_sampler =SubsetRandomSampler(train_idx)
valid_sampler =SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=2)
validloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=2)

In [ ]:
p=[]
s0=0
length=0
for i,l in trainloader:
    p=p+l.tolist()
traindf=pd.DataFrame(p,columns=['traintarget'])

In [ ]:
p1=[]
s0=0
length=0
for i,l in validloader:
    p1=p1+l.tolist()
validdf=pd.DataFrame(p1,columns=['validtarget'])

In [ ]:
traindf.hist()
validdf.hist()
data.hist()

In [ ]:
LEARNING_RATE = 0.001

#weights = [1, 0.25]
#class_weights = torch.FloatTensor(weights).cuda()

criterion = torch.nn.CrossEntropyLoss(
    #weight=class_weights
)
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
model.to(device)

In [ ]:
def createDirectory(dirname):
    try:
        os.mkdir(dirname)
    except FileExistsError:
        print("Directory " + dirname + " already exists.")
models_dir = "models/"
createDirectory(models_dir)

total_epoch = 20

best_epoch = 0
training_losses = []
val_losses = []


for epoch in range(total_epoch):
    
    epoch_train_loss = 0
    pred=[]
    target=[]
    
    for X, y in trainloader:
        
        X, y = X.cuda(), y.cuda()
        
        optimizer.zero_grad()
        result = model(X)
        loss = criterion(result, y)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
      
    training_losses.append(epoch_train_loss)
    
    
    epoch_val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X, y in validloader:
            
            X, y = X.cuda(), y.cuda()
             
            result = model(X)
            loss = criterion(result, y)
            epoch_val_loss += loss.item()
            
            
        
            _, maximum = torch.max(result.data, 1)
            maximum=maximum.tolist()
            y=y.tolist()
            pred = pred+maximum
            target = target+y
            #total += y.size(0)
            #correct += (maximum == y).sum().item()
            
    val_losses.append(epoch_val_loss)
    
    import numpy as np
    from sklearn.metrics import roc_auc_score

    #pred = np.vstack((pred)).ravel()
    #auc = roc_auc_score(target, pred)
    
    #accuracy = correct/total
    print("EPOCH:", epoch, ", Training Loss:", epoch_train_loss, ", Validation Loss:", epoch_val_loss)
    
    
    if min(val_losses) == val_losses[-1]:
        best_epoch = epoch
        checkpoint = {'model': model,
                            'state_dict': model.state_dict(),
                            'optimizer' : optimizer.state_dict()}

        torch.save(checkpoint, models_dir + '{}.pth'.format(epoch))
        print("Model saved")

In [ ]:
plt.plot(range(total_epoch), training_losses, label='Training')
plt.plot(range(total_epoch), val_losses, label='Validation')
plt.legend()

In [ ]:
models_dir = "models/"

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    return model.eval()


filepath = models_dir + str(best_epoch) + ".pth"
loaded_model = load_checkpoint(filepath)

In [ ]:
from torch.nn import functional as F

from PIL import Image
test_img=[]
pred=[]
for img in sub['FILE'] :
      img_path=root_path + dirts +str(img)
      test_image = Image.open(img_path).convert('RGB')

      test_image_tensor = train_transforms(test_image)
      test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

      test_image_tensor=test_image_tensor.cuda()

      with torch.no_grad():
          result = loaded_model(test_image_tensor)
          result=F.softmax(result,dim=1).data
          result = result.squeeze(0)
          #result=torch.exp(result)
          result=result.tolist()
          pred.append(result)

# test loader

In [ ]:
pred=np.array(pred)
dict = {}
for col in classes:
    dict[col] = pred[:,label_map[col]]
submissions = pd.DataFrame(dict, columns=classes)

test_df = pd.DataFrame({
    'FILE': sub['FILE'].values
})

submission = pd.concat([test_df, submissions], axis=1)

In [ ]:
!pip install --upgrade efficientnet-pytorch
from efficientnet_pytorch import EfficientNet
class EfficientNetwork(nn.Module):
    def __init__(self, output_size, b1=False, b2=False):
        super().__init__()
        self.b1, self.b2 = b1, b2
        
        # Define Feature part
        if b1:
            self.features = EfficientNet.from_pretrained('efficientnet-b1')
        elif b2:
            self.features = EfficientNet.from_pretrained('efficientnet-b2')
        else:
            self.features = EfficientNet.from_pretrained('efficientnet-b0')
        
        # Define Classification part
        if b1:
            self.classification = nn.Linear(1280, output_size)
        elif b2:
            self.classification = nn.Linear(1408, output_size)
        else:
            self.classification = nn.Linear(1280, output_size)
        
        
    def forward(self, image, prints=False):
        if prints: print('Input Image shape:', image.shape)
        
        image = self.features.extract_features(image)
        if prints: print('Features Image shape:', image.shape)
            
        if self.b1:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1280)
        elif self.b2:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1408)
        else:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1280)
        if prints: print('Image Reshaped shape:', image.shape)
        
        out = self.classification(image)
        if prints: print('Out shape:', out.shape)
        
        return out

# Create an example model (B2)
model = EfficientNetwork(output_size=137, b1=False, b2=True)

In [ ]:
model.to(device)

In [ ]:
LEARNING_RATE = 0.001

#weights = [1, 0.25]
#class_weights = torch.FloatTensor(weights).cuda()

criterion = torch.nn.CrossEntropyLoss(
    #weight=class_weights
)
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
import shutil

shutil.rmtree('/kaggle/working/models',ignore_errors=True)

In [ ]:
def createDirectory(dirname):
    try:
        os.mkdir(dirname)
    except FileExistsError:
        print("Directory " + dirname + " already exists.")
models_dir = "models/"
createDirectory(models_dir)

total_epoch = 20

best_epoch = 0
training_losses = []
val_losses = []


for epoch in range(total_epoch):
    
    epoch_train_loss = 0
    pred=[]
    target=[]
    
    for X, y in trainloader:
        
        X, y = X.cuda(), y.cuda()
        
        optimizer.zero_grad()
        result = model(X)
        loss = criterion(result, y)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
      
    training_losses.append(epoch_train_loss)
    
    
    epoch_val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X, y in validloader:
            
            X, y = X.cuda(), y.cuda()
             
            result = model(X)
            loss = criterion(result, y)
            epoch_val_loss += loss.item()
            
            
        
            _, maximum = torch.max(result.data, 1)
            maximum=maximum.tolist()
            y=y.tolist()
            pred = pred+maximum
            target = target+y
            #total += y.size(0)
            #correct += (maximum == y).sum().item()
            
    val_losses.append(epoch_val_loss)
    
    import numpy as np
    from sklearn.metrics import roc_auc_score

    #pred = np.vstack((pred)).ravel()
    #auc = roc_auc_score(target, pred)
    
    #accuracy = correct/total
    print("EPOCH:", epoch, ", Training Loss:", epoch_train_loss, ", Validation Loss:", epoch_val_loss)
    
    
    if min(val_losses) == val_losses[-1]:
        best_epoch = epoch
        checkpoint = {'model': model,
                            'state_dict': model.state_dict(),
                            'optimizer' : optimizer.state_dict()}

        torch.save(checkpoint, models_dir + '{}.pth'.format(epoch))
        print("Model saved")

In [ ]:
plt.plot(range(total_epoch), training_losses, label='Training')
plt.plot(range(total_epoch), val_losses, label='Validation')
plt.legend()

In [ ]:
models_dir = "models/"

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    return model.eval()


filepath = models_dir + str(best_epoch) + ".pth"
loaded_model = load_checkpoint(filepath)


In [ ]:
from torch.nn import functional as F

from PIL import Image
test_img=[]
pred=[]
for img in sub['FILE'] :
      img_path=root_path + dirts +str(img)
      test_image = Image.open(img_path).convert('RGB')

      test_image_tensor = train_transforms(test_image)
      test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

      test_image_tensor=test_image_tensor.cuda()

      with torch.no_grad():
          result = loaded_model(test_image_tensor)
          result=F.softmax(result,dim=1).data
          result = result.squeeze(0)
          #result=torch.exp(result)
          result=result.tolist()
          pred.append(result)

In [ ]:
pred=np.array(pred)
dict = {}
for col in classes:
    dict[col] = pred[:,label_map[col]]
submissions1 = pd.DataFrame(dict, columns=classes)

test_df = pd.DataFrame({
    'FILE': sub['FILE'].values
})

submission1 = pd.concat([test_df, submissions1], axis=1)

Ensembling

In [ ]:
for i in submission.columns[1:]:
    finalpredictions[i]=(submission[i] + submission1[i]) / 2